In [22]:
import os
from multiprocessing import cpu_count
import numpy as np
import paddle
import paddle.fluid as fluid
import matplotlib.pyplot as plt
paddle.enable_static()
data_root_path='./data/'
 

# 创建数据集
def create_data_list(data_root_path):
    with open(data_root_path + 'dev_list.txt', 'w') as f:
        pass
    with open(data_root_path + 'train_list.txt', 'w') as f:
        pass

    with open(os.path.join(data_root_path, 'dict_txt.txt'), 'r', encoding='utf-8') as f_data:
        dict_txt = eval(f_data.readlines()[0])

    with open(data_root_path + 'train.txt', 'r', encoding='utf-8') as f_data:
        lines = f_data.readlines()

    with open(os.path.join(data_root_path, 'dev.txt'), 'r', encoding='utf-8') as f_data:
        lines = f_data.readlines()

    for line in lines:
        title = line.split('\t')[0]
        l = line.split('\t')[-1].replace('\n', '')
        l = dict_reverse[l]
        labs = ""
        if True:
            with open(os.path.join(data_root_path, 'dev_list.txt'), 'a', encoding='utf-8') as f_test:
                for s in title:
                    lab = str(dict_txt[s])
                    labs = labs + lab + ','
                labs = labs[:-1]
                labs = labs + '\t' + str(l) + '\n'
                f_test.write(labs)

    with open(os.path.join(data_root_path, 'train.txt'), 'r', encoding='utf-8') as f_data:
        lines = f_data.readlines()

    for line in lines:
        title = line.split('\t')[0]
        l = line.split('\t')[-1].replace('\n', '')
        l = dict_reverse[l]
        labs = ""
        if True:
            with open(os.path.join(data_root_path, 'train_list.txt'), 'a', encoding='utf-8') as f_test:
                for s in title:
                    lab = str(dict_txt[s])
                    labs = labs + lab + ','
                labs = labs[:-1]
                labs = labs + '\t' + str(l) + '\n'
                f_test.write(labs)
    print("数据列表生成完成！")

Y_dict = {0: '财经',
 1: '彩票',
 2: '房产',
 3: '股票',
 4: '家居',
 5: '教育',
 6: '科技',
 7: '社会',
 8: '时尚',
 9: '时政',
 10: '体育',
 11: '星座',
 12: '游戏',
 13: '娱乐'}

dict_list = []

for s in Y_dict:
    dict_list.append([Y_dict[s], s])
# 添加未知字符
dict_reverse = dict(dict_list)
end_dict = {"<unk>": 14}
dict_reverse.update(end_dict)
# 把这些字典保存到本地中
with open(os.path.join(data_root_path, 'dict.txt'), 'w', encoding='utf-8') as f:
    f.write(str(dict_reverse))


# 把下载得数据生成一个字典
def create_dict(data_path, dict_path):
    dict_set = set()
    # 统计有多少种类别，分别对应的id,并显示出来以供后面的预测应用
    id_and_className = {}

    # 读取已经下载得数据
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # 把数据生成一个元组
    for line in lines:
        lineList = line.split('\t')
        classId = lineList[-1].replace('\n', '')
        title = lineList[0]
        if classId not in id_and_className.keys():
            id_and_className[classId] = lineList[1]

        for s in title:
            dict_set.add(s)
    # 把元组转换成字典，一个字对应一个数字
    dict_list = []
    i = 0
    for s in dict_set:
        dict_list.append([s, i])
        i += 1
    # 添加未知字符
    dict_txt = dict(dict_list)
    end_dict = {"<unk>": i}
    dict_txt.update(end_dict)
    # 把这些字典保存到本地中
    with open(dict_path, 'w', encoding='utf-8') as f:
        f.write(str(dict_txt))
    print("数据字典生成完成！")
    print('类Id及其类别名称:', id_and_className)


# 获取字典的长度
def get_dict_len(dict_path):
    with open(dict_path, 'r', encoding='utf-8') as f:
        line = eval(f.readlines()[0])
 
    return len(line.keys())
def data_mapper(sample):
   data, label = sample
   dataList=[]
   for e in data.split(','):
        if e=='':
           print('meet blank')
        else:
            dataList.append(np.int64(e))
   return dataList, int(label)
 
# 创建数据读取器train_reader
def train_reader(train_list_path):
   def reader():
       with open(train_list_path, 'r') as f:
           lines = f.readlines()
           # 打乱数据
           np.random.shuffle(lines)
           # 开始获取每张图像和标签
           for line in lines:
               data, label = line.split('\t')
               yield data, label
   return paddle.reader.xmap_readers(data_mapper, reader, cpu_count(), 1024)
#  创建数据读取器test_reader
def test_reader(test_list_path):
   def reader():
       with open(test_list_path, 'r') as f:
           lines = f.readlines()
           for line in lines:
               data, label = line.split('\t')
               yield data, label
   return paddle.reader.xmap_readers(data_mapper, reader, cpu_count(), 1024)
 



In [23]:
#   网络定义
def CNN_net(data, dict_dim, class_dim=14, emb_dim=128, hid_dim=128, hid_dim2=98):
    emb = fluid.layers.embedding(input=data,size=[dict_dim, emb_dim])
    conv_1 = fluid.nets.sequence_conv_pool(
           input=emb,
           num_filters=hid_dim,
           filter_size=3,
           act="tanh",
           pool_type="max")
    conv_2 = fluid.nets.sequence_conv_pool(
           input=emb,
           num_filters=hid_dim,
           filter_size=4,
           act="tanh",
           pool_type="max")
    conv_3 = fluid.nets.sequence_conv_pool(
            input=emb,
            num_filters=hid_dim2,
            filter_size=4,
            act="tanh",
            pool_type="max")
    fc1 = fluid.layers.fc(input=[conv_1, conv_2,conv_3], size=128, act='softmax')
    bn = fluid.layers.batch_norm(input=fc1, act='relu')
    fc2 = fluid.layers.fc(input= bn, size=64, act='softmax')
    bn1 = fluid.layers.batch_norm(input=fc2, act='relu')
    fc3 = fluid.layers.fc(input= bn1, size=class_dim, act='softmax')
    return fc3
 
# 定义绘制训练过程的损失值和准确率变化趋势的方法draw_train_process
all_train_iter=0
all_train_iters=[]
all_train_costs=[]
all_train_accs=[]
def draw_train_process(title,iters,costs,accs,label_cost,lable_acc):
    plt.title(title, fontsize=24)
    plt.xlabel("iter", fontsize=20)
    plt.ylabel("cost/acc", fontsize=20)
    plt.plot(iters, costs,color='red',label=label_cost)
    plt.plot(iters, accs,color='green',label=lable_acc)
    plt.legend()
    plt.grid()
    plt.show()
 
# 把生产的数据列表都放在自己的总类别文件夹中
data_path = os.path.join(data_root_path, 'train.txt')
dict_path = os.path.join(data_root_path, "dict_txt.txt")
# 创建数据字典
create_dict(data_path, dict_path)
# 创建数据列表
create_data_list(data_root_path)
 
words = fluid.layers.data(name='words', shape=[1], dtype='int64', lod_level=1)
label = fluid.layers.data(name='label', shape=[1], dtype='int64')
# 获取数据字典长度
dict_dim = get_dict_len('./data/dict_txt.txt')
# 获取卷积神经网络
# 获取分类器
model = CNN_net(words, dict_dim)
# 获取损失函数和准确率
cost = fluid.layers.cross_entropy(input=model, label=label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input=model, label=label)
 
# 获取预测程序
test_program = fluid.default_main_program().clone(for_test=True)
# 定义优化方法
optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.002)
opt = optimizer.minimize(avg_cost)
 
# 创建一个执行器，CPU训练速度比较慢
# 定义使用CPU还是GPU，使用CPU时use_cuda = False,使用GPU时use_cuda = True
use_cuda =  False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
#place = fluid.CPUPlace()
#place = fluid.CUDAPlace(0)
exe = fluid.Executor(place)

数据字典生成完成！
类Id及其类别名称: {'科技': '科技\n', '体育': '体育\n', '时政': '时政\n', '股票': '股票\n', '娱乐': '娱乐\n', '教育': '教育\n', '家居': '家居\n', '财经': '财经\n', '房产': '房产\n', '社会': '社会\n', '游戏': '游戏\n', '彩票': '彩票\n', '星座': '星座\n', '时尚': '时尚\n'}
数据列表生成完成！


In [24]:
# 进行参数初始化
exe.run(fluid.default_startup_program())
train_reader = paddle.batch(reader=train_reader('./data/train_list.txt'), batch_size=128)
test_reader = paddle.batch(reader=test_reader('./data/dev_list.txt'), batch_size=128)
feeder = fluid.DataFeeder(place=place, feed_list=[words, label])
 
EPOCH_NUM=1
model_save_dir = './infer_model/'
# 开始训练
 
for pass_id in range(EPOCH_NUM):
   # 进行训练
   for batch_id, data in enumerate(train_reader()):
       train_cost, train_acc = exe.run(program=fluid.default_main_program(),
                            feed=feeder.feed(data),
                            fetch_list=[avg_cost, acc])
       all_train_iter = all_train_iter + 100
       all_train_iters.append(all_train_iter)
       all_train_costs.append(train_cost[0])
       all_train_accs.append(train_acc[0])
       if batch_id % 100 == 0:
           print('Train Pass:%d, Batch:%d, Cost:%0.5f, Acc:%0.5f' % (pass_id, batch_id, train_cost[0], train_acc[0]))
   # 进行测试
   test_costs = []
   test_accs = []
   for batch_id, data in enumerate(test_reader()):
       test_cost, test_acc = exe.run(program=test_program,
                                             feed=feeder.feed(data),
                                             fetch_list=[avg_cost, acc])
       test_costs.append(test_cost[0])
       test_accs.append(test_acc[0])
   # 计算平均预测损失在和准确率
   test_cost = (sum(test_costs) / len(test_costs))
   test_acc = (sum(test_accs) / len(test_accs))
   print('Test:%d, Cost:%0.5f, ACC:%0.5f' % (pass_id, test_cost, test_acc))
 
if not os.path.exists(model_save_dir):
   os.makedirs(model_save_dir)
fluid.io.save_inference_model(os.path.join(save_dir=model_save_dir, 'model_state.pdparams'),
                           feeded_var_names=[words.name],
                           target_vars=[model],
                           executor=exe)
print('训练模型保存完成！')
draw_train_process("training", all_train_iters, all_train_costs, all_train_accs, "trainning cost", "trainning acc")

Train Pass:0, Batch:0, Cost:2.63319, Acc:0.03906
Train Pass:0, Batch:100, Cost:1.16956, Acc:0.69531
Train Pass:0, Batch:200, Cost:0.81127, Acc:0.80469
Train Pass:0, Batch:300, Cost:0.77494, Acc:0.79688
Train Pass:0, Batch:400, Cost:0.88253, Acc:0.72656
Train Pass:0, Batch:500, Cost:0.69675, Acc:0.75781
Train Pass:0, Batch:600, Cost:0.74678, Acc:0.74219
Train Pass:0, Batch:700, Cost:0.49288, Acc:0.85938
Train Pass:0, Batch:800, Cost:0.62903, Acc:0.79688
Train Pass:0, Batch:900, Cost:0.53129, Acc:0.89062
Train Pass:0, Batch:1000, Cost:0.59342, Acc:0.85938
Train Pass:0, Batch:1100, Cost:0.56626, Acc:0.85156
Train Pass:0, Batch:1200, Cost:0.54455, Acc:0.82812
Train Pass:0, Batch:1300, Cost:0.70999, Acc:0.80469
Train Pass:0, Batch:1400, Cost:0.44355, Acc:0.89844
Train Pass:0, Batch:1500, Cost:0.55202, Acc:0.82812
Train Pass:0, Batch:1600, Cost:0.31157, Acc:0.94531
Train Pass:0, Batch:1700, Cost:0.55303, Acc:0.81250
Train Pass:0, Batch:1800, Cost:0.59024, Acc:0.82031
Train Pass:0, Batch:1900

NameError: name 'save_dir' is not defined

In [ ]:
# 加载在验证集上效果最优的一轮的模型参数
import os
import paddle

params_path = './infer_model/model_state.pdparams'
if params_path and os.path.isfile(params_path):
    # 加载模型参数
    state_dict = paddle.load(params_path)
    model.set_dict(state_dict)
    print("Loaded parameters from %s" % params_path)

    # 测试最优模型参数在验证集上的分数
evaluate(model, criterion, metric, dev_data_loader)